# Simple Word2vec model

Imports and Data Preparation

In [ ]:
from numpy.random import seed
seed(1)
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from nltk.tokenize import sent_tokenize,word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import nltk
from nltk.corpus import stopwords

from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import tensorflow as tf
from tensorflow.keras.layers import LSTM
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import StandardScaler

import pickle

nltk.download('punkt')
nltk.download('stopwords')
random_state = 7

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
train = pd.read_csv('/content/drive/My Drive/Author_identification/train.csv')
test = pd.read_csv('/content/drive/My Drive/Author_identification/test.csv')

Mounted at /content/drive


In [ ]:
train['sentences'] = train.text.transform(lambda x: len(sent_tokenize(x)))
train['words'] = train.text.transform(lambda x: len(word_tokenize(x)))
train['text_length'] = train.text.transform(lambda x: len(x))

text_info = train.groupby("author")[['sentences','words','text_length']].sum()
text_info

,sentences,words,text_length
author,,,
EAP,8206,232184,1123585
HPL,5876,173979,878178
MWS,6128,188824,916632


In [ ]:
author_dict = {'EAP': 0, 'HPL': 1, 'MWS': 2}
new_author = []
new_text_ws = [] # with stopwords
for i, row in train[['text','author']].iterrows():
    word_tokens = word_tokenize(row['text'])
    new_text_ws.append(" ".join([w for w in word_tokens if not w in [',','.','?','!',':',';',"'",'"','-',"''","`","``"]]).lower())
    new_author.append(author_dict[row['author']])
new_train_ws = pd.DataFrame(data={'text': new_text_ws, 'author': new_author})

Word2vec vectors for each author

In [ ]:
#X_train, X_valid = train_test_split(data_0, test_size=0.3, random_state=random_state)

In [ ]:
import multiprocessing
from gensim.models import Word2Vec

data_0 = new_train_ws.loc[new_train_ws['author'] == 0]
data_1 = new_train_ws.loc[new_train_ws['author'] == 1]
data_2 = new_train_ws.loc[new_train_ws['author'] == 2]

data_0 = data_0.sample(frac=1).reset_index(drop=True)
data_1 = data_1.sample(frac=1).reset_index(drop=True)
data_2 = data_2.sample(frac=1).reset_index(drop=True)

train_0, test_0 = train_test_split(data_0, test_size=0.1, random_state=random_state)
train_1, test_1 = train_test_split(data_1, test_size=0.1, random_state=random_state)
train_2, test_2 = train_test_split(data_2, test_size=0.1, random_state=random_state)

In [ ]:
min_count = 5

In [ ]:
from gensim.models.phrases import Phrases, Phraser
sent_0 = [row.split() for row in train_0['text']]
phrases_0 = Phrases(sent_0, min_count=min_count, progress_per=10000)
bigram_0 = Phraser(phrases_0)
sentences_0 = bigram_0[sent_0]
w2v_model_0 = Word2Vec(min_count=min_count, window=2, size=300, sample=6e-5, alpha=0.03, min_alpha=0.0007, negative=20, workers=2)

from time import time
t = time()
w2v_model_0.build_vocab(sentences_0, progress_per=10000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

t = time()
w2v_model_0.train(sentences_0, total_examples=w2v_model_0.corpus_count, epochs=30, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

w2v_model_0.init_sims(replace=True)

w2v_model_0.wv.similarity('you', 'are')

Time to build vocab: 0.02 mins
Time to train the model: 0.4 mins


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.9255816

In [ ]:
from gensim.models.phrases import Phrases, Phraser
sent_1 = [row.split() for row in train_1['text']]
phrases_1 = Phrases(sent_1, min_count=min_count, progress_per=10000)
bigram_1 = Phraser(phrases_1)
sentences_1 = bigram_1[sent_1]
w2v_model_1 = Word2Vec(min_count=min_count, window=2, size=300, sample=6e-5, alpha=0.03, min_alpha=0.0007, negative=20, workers=2)

from time import time
t = time()
w2v_model_1.build_vocab(sentences_1, progress_per=10000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

t = time()
w2v_model_1.train(sentences_1, total_examples=w2v_model_1.corpus_count, epochs=30, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

w2v_model_1.init_sims(replace=True)

w2v_model_1.wv.similarity('you', 'are')

Time to build vocab: 0.02 mins
Time to train the model: 0.3 mins


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.9961437

In [ ]:
from gensim.models.phrases import Phrases, Phraser
sent_2 = [row.split() for row in train_2['text']]
phrases_2 = Phrases(sent_2, min_count=min_count, progress_per=10000)
bigram_2 = Phraser(phrases_2)
sentences_2 = bigram_2[sent_2]
w2v_model_2 = Word2Vec(min_count=min_count, window=2, size=300, sample=6e-5, alpha=0.03, min_alpha=0.0007, negative=20, workers=2)

from time import time
t = time()
w2v_model_2.build_vocab(sentences_2, progress_per=10000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

t = time()
w2v_model_2.train(sentences_2, total_examples=w2v_model_2.corpus_count, epochs=30, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

w2v_model_2.init_sims(replace=True)

w2v_model_2.wv.similarity('you', 'are')

Time to build vocab: 0.02 mins
Time to train the model: 0.32 mins


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.9804925

In [ ]:
word_vectors_0 = w2v_model_0.wv
word_vectors_1 = w2v_model_1.wv
word_vectors_2 = w2v_model_2.wv

Testing function: 
- input: test sentence
- output: the most likely author

In [ ]:
def testing(X,y):
  true = 0
  false = 0
  for index, a in X.items():
    #print(a)

    sims = []
    tokens = word_tokenize(a)
    lenght = len(tokens)

    sim_0 = 0.5*(lenght-1)
    sim_1 = 0.5*(lenght-1)
    sim_2 = 0.5*(lenght-1)
    for i in range(lenght-1):
      #print(i)
      if tokens[i] in word_vectors_0 and tokens[i+1] in word_vectors_0:
        sim = w2v_model_0.wv.similarity(tokens[i], tokens[i+1])
        sim_0 += sim-0.5   

      if tokens[i] in word_vectors_1 and tokens[i+1] in word_vectors_1:
        sim = w2v_model_1.wv.similarity(tokens[i], tokens[i+1])
        sim_1 += sim-0.5

      if tokens[i] in word_vectors_2 and tokens[i+1] in word_vectors_2:
        sim = w2v_model_2.wv.similarity(tokens[i], tokens[i+1])
        sim_2 += sim-0.5

    #print((sim_0,sim_1,sim_2))
    sims.append(sim_0/(lenght-1))
    sims.append(sim_1/(lenght-1))
    sims.append(sim_2/(lenght-1))
    
    #print(sims)
    #print(np.argmax(sims),y[index])
    if np.argmax(sims) == y[index]:
      true += 1
    else:
      false += 1
  print(true/(true+false))
  return (true, false)

In [ ]:
true_0, false_0 = testing(test_0['text'],test_0['author'])
true_1, false_1 = testing(test_1['text'],test_1['author'])
true_2, false_2 = testing(test_2['text'],test_2['author'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.5088607594936709
0.7783687943262412
0.7487603305785124


In [ ]:
true = true_0+ true_1 +true_2
false = false_0 + false_1 + false_2
print(true/(true+false))

0.6605410923940787


So, this method giving an accuracy of 66.05%.